# Neural network with pytorch

In [4]:
!nvidia-smi

Sat Apr  6 13:09:41 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.78                 Driver Version: 551.78         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   56C    P8              3W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor,Lambda,Compose
import matplotlib.pyplot as plt

In [8]:
train_data=datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data=datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [10]:
train_dataloader=DataLoader(train_data,batch_size=64)
test_dataloader=DataLoader(test_data,batch_size=64)
for X,y in test_dataloader:
    print('Shape of X [N,C,H,W]: ',X.shape )
    print('Shape of y: ',y.shape,y.dtype)
    break

Shape of X [N,C,H,W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [13]:
device='cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self,x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

model=NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [15]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=1e-3)

In [22]:
def train(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y=X.to(device),y.to(device)
        pred=model(X)
        loss=loss_fn(pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 ==0:
            loss,current=loss.item(),batch*len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [23]:
def test(dataloader,model,loss_fn):
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    model.eval()
    test_loss,correct=0,0
    with torch.no_grad():
        for X,y  in dataloader:
            X,y=X.to(device),y.to(device)
            pred=model(X)
            test_loss+=loss_fn(pred,y).item()
            correct +=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss/=num_batches
    correct/=size
    print(f'Test error: \n Accuracy: {(100*correct):>0.1f}%,Avg loss: {test_loss:>8f} \n')

In [24]:
epochs=5
for e in range(epochs):
    print(f'Epoch {e+1}\n-----------------------------')
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
print('Done!')

Epoch 1
-----------------------------
loss: 2.290448 [    0/60000]
loss: 2.278569 [ 6400/60000]
loss: 2.257871 [12800/60000]
loss: 2.259960 [19200/60000]
loss: 2.243860 [25600/60000]
loss: 2.221109 [32000/60000]
loss: 2.227580 [38400/60000]
loss: 2.197525 [44800/60000]
loss: 2.179141 [51200/60000]
loss: 2.163313 [57600/60000]
Test error: 
 Accuracy: 43.0%,Avg loss: 2.147549 

Epoch 2
-----------------------------
loss: 2.149966 [    0/60000]
loss: 2.136656 [ 6400/60000]
loss: 2.078164 [12800/60000]
loss: 2.098691 [19200/60000]
loss: 2.045112 [25600/60000]
loss: 1.990762 [32000/60000]
loss: 2.017485 [38400/60000]
loss: 1.944234 [44800/60000]
loss: 1.929263 [51200/60000]
loss: 1.872710 [57600/60000]
Test error: 
 Accuracy: 56.9%,Avg loss: 1.860938 

Epoch 3
-----------------------------
loss: 1.892295 [    0/60000]
loss: 1.858322 [ 6400/60000]
loss: 1.744107 [12800/60000]
loss: 1.782758 [19200/60000]
loss: 1.675579 [25600/60000]
loss: 1.635528 [32000/60000]
loss: 1.656990 [38400/60000]
l